In [ ]:
import json
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(6,2)})
sns.set_style("white")

def read_ldjson(filename):
    res = []
    with open(filename) as xfp:
        for l in xfp:
            res.append(json.loads(l))
    return res

In [ ]:
# which cpu
!cat /proc/cpuinfo | grep "model name" | head -n 1

In [ ]:
# test run of preset script
!./avgtime.py 3 ./est_para.sh fast 26 2.mp4

In [ ]:
! rm preset.ldjson
for rep in range(20):
    for preset in ["ultrafast", "superfast", "veryfast", "faster", "fast", "medium", "slow"]:
        !./avgtime.py 1 ./est_para.sh {preset} 26 2.mp4 | grep "cmd" >> preset.ldjson

In [ ]:
dpreset = pd.DataFrame(read_ldjson("preset.ldjson"))
dpreset

In [ ]:
ax = sns.boxplot(x="avgtime", y="arg_1", data=dpreset)
ax.set_title("preset variable; crf=26; 20 repetitions")
ax.set_xlabel("average time [s]")
ax.set_ylabel("preset")
ax.get_figure().savefig("preset.pdf", bbox_inches="tight")

In [ ]:
print("processing time [s] for medium", dpreset[dpreset["arg_1"] == "medium"]["avgtime"].mean())

In [ ]:
print("processing time [s] for superfast", dpreset[dpreset["arg_1"] == "superfast"]["avgtime"].mean())

In [ ]:
# run crf script
! rm crf.ldjson
for rep in range(20):
    for crf in [20, 22, 24, 26, 28, 30, 32, 34]:
        !./avgtime.py 1 ./est_para.sh medium {crf} 2.mp4 | grep "cmd" >> crf.ldjson
    

In [ ]:
dcrf = pd.DataFrame(read_ldjson("crf.ldjson"))
dcrf

In [ ]:
ax = sns.boxplot(x="avgtime", y="arg_2", data=dcrf)
ax.set_title("medium preset; crf variable; 20 repetitions")
ax.set_xlabel("average time [s]")
ax.set_ylabel("crf")
ax.get_figure().savefig("crf.pdf", bbox_inches="tight")


In [ ]:
print("processing time [s] for crf26", dcrf[dcrf["arg_2"] == "26"]["avgtime"].mean())

In [ ]:
print("processing time [s] for crf28", dcrf[dcrf["arg_2"] == "28"]["avgtime"].mean())

In [ ]:
# run of frame sampling script; -crf 26 -preset medium
! rm framesampling.ldjson
for rep in range(20):
    for framesampling in [5, 10, 20, 30, 60]:
        !./avgtime.py 1 ./est_framesampling.sh {framesampling} 2.mp4 | grep "cmd" >> framesampling.ldjson

In [ ]:
dframesampling = pd.DataFrame(read_ldjson("framesampling.ldjson"))
dframesampling

In [ ]:
ax = sns.boxplot(x="avgtime", y="arg_1", data=dframesampling)
ax.set_title("framesampling with medium preset; crf=26; 20 repetitions")
ax.set_xlabel("average time [s]")
ax.set_ylabel("interval")
ax.get_figure().savefig("framesampling.pdf", bbox_inches="tight")

In [ ]:
! rm -rf modelX.ldjson
for rep in range(20):
    !./avgtime.py 1 ./../models/est_sf_crf.sh 2.mp4 | grep "cmd" >> modelX.ldjson

In [ ]:
dX = pd.DataFrame(read_ldjson("modelX.ldjson"))
dX

In [ ]:
dX["avgtime"].mean(), dX["avgtime"].std() 

In [ ]:
! rm -rf modelY.ldjson
for rep in range(20):
    !./avgtime.py 1 ./../models/est_uf_qp_sr_crf.sh 2.mp4 | grep "cmd" >> modelY.ldjson

In [ ]:
dY = pd.DataFrame(read_ldjson("modelY.ldjson"))
dY

In [ ]:
dY["avgtime"].mean(), dY["avgtime"].std()